In [ ]:
import numpy as np
import pandas as pd
import glob
import string
import re
import nltk

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from textwrap import wrap

# constants
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
'''
Merge datasets in the data folder.
Returns a dataframe of all the data.
'''
def retrieve_reviews_df():
    all_files = glob.glob("../data/*.csv")  

    df_list = []

    for filename in all_files:
        print(f"Concatenating {filename}")
        df = pd.read_csv(filename, index_col=None, header=0)
        df_list.append(df)
    
    return pd.concat(df_list, axis=0, ignore_index=True)

In [5]:
# preprocessing for the dataframe 
def score(x):
    if x<3:
        return -1
    elif x==3 or x==4:
        return 0
    else:
        return 1

def preprocessing(df):
    # drop columns
    new_df = df.drop(columns = ["ProductId", "UserId", "ProfileName", "HelpfulnessNumerator", "HelpfulnessDenominator", "Time"])

    # drop na values
    new_df = new_df.dropna(axis=0)

    # make a new column of sentiment: (-1/0/1) -- pos/neutral/neg -- 1,2/3/4,5
    new_df['Sentiment'] = new_df.apply(lambda x: score(x['Score']), axis=1)


    return new_df

def retrieve_and_preprocess():
    # Create one dataframe with data, then drop columns and change score to 1/0/-1
    df = preprocessing(retrieve_reviews_df())

    # Save df to csv?

    return df

'0.23.1'

In [ ]:
# helper functions for text_preprocess
def remove_html_tags(text):
    no_html = re.sub('<.*?>','',text)
    return no_html

def remove_stopwords(text):
    products = ['dog', 'food', 'soup', 'chai', 'tea','ordered','order','coconut','taffy','product']
    output = " ".join([i for i in text.split() if i not in stopwords and i not in products])
    return output

def remove_punctuation(text):
    punctuation_free = "".join([i for i in text if i not in string.punctuation])
    return punctuation_free

In [ ]:
'''
Options:
- html -- remove html headers and syntax
- stop -- remove stopwords
- punc -- remove punctuation
- lower -- change entire string to lowercase
'''
def text_preprocess(df, options):
    df["Clean_text"] = df["Text"]
    
    if 'html' in options:
        df["Clean_text"] = df["Clean_text"].apply(lambda x: remove_html_tags(x))

    if 'stop' in options:
        df["Clean_text"] = df["Clean_text"].apply(lambda x: remove_stopwords(x))

    if 'punc' in options:
        df["Clean_text"] = df["Clean_text"].apply(lambda x: remove_punctuation(x))

    if 'lower' in options:
        df["Clean_text"] = df["Clean_text"].apply(lambda x: x.lower())

In [ ]:
'''
X_train : should be a series, df['Text'] (according to EDA ipynb)
'''
def vectorize(X):
    # tokenizer
    count_vect = CountVectorizer()
    X_counts = count_vect.fit_transform(X) 

    # tf-idf
    tfidf_transformer = TfidfTransformer(use_idf=False)
    X_tfidf = tfidf_transformer.fit_transform(X_counts)


    return X_counts, X_tfidf